In [1]:
import numpy as np
import pandas as pd

In [2]:
data=[['Sunny','Hot','High','Weak',0],['Sunny','Hot','High','Strong',0],['Overcast','Hot','High','Weak',1],['Rain','Mild','High','Weak',1],\
      ['Rain','Cool','Normal','Weak',1],['Rain','Cool','Normal','Strong',0],['Overcast','Cool','Normal','Strong',1],['Sunny','Mild','High','Weak',0],\
      ['Sunny','Cool','Normal','Weak',1],['Rain','Mild','Normal','Weak',1],['Sunny','Mild','Normal','Strong',1],['Overcast','Mild','High','Strong',1],\
      ['Overcast','Hot','Normal','Weak',1],['Rain','Mild','High','Strong',0]]
Data=pd.DataFrame(data,columns=['Outlook','Temperature','Humidity','Wind','PlayTennis'])

In [3]:
Data.head()

,Outlook,Temperature,Humidity,Wind,PlayTennis
0,Sunny,Hot,High,Weak,0
1,Sunny,Hot,High,Strong,0
2,Overcast,Hot,High,Weak,1
3,Rain,Mild,High,Weak,1
4,Rain,Cool,Normal,Weak,1


In [4]:
cols=Data.shape[1]
X_data=Data.iloc[:,:cols-1]
Y_data=Data.iloc[:,cols-1:]
featureNames=X_data.columns

In [5]:
def Naive_Bayes(X_data,Y_data):
    y=Y_data.values
    X=X_data.values
    y_unique=np.unique(y)
    prior_prob=np.zeros(len(y_unique))
    for i in range(len(y_unique)):
        prior_prob[i]=sum(y==y_unique[i])/len(y)

    condition_prob={}

    for feat in featureNames:
        x_unique=list(set(X_data[feat]))
        x_condition_prob=np.zeros((len(y_unique),len(x_unique)))
        for j in range(len(y_unique)):
            for k in range(len(x_unique)):
                x_condition_prob[j,k]=\
                sum((X_data[feat]==x_unique[k])&(Y_data.PlayTennis==y_unique[j]))/sum(y==y_unique[j])
        x_condition_prob=pd.DataFrame(x_condition_prob,columns=x_unique,index=y_unique)
        condition_prob[feat]=x_condition_prob
    return prior_prob,condition_prob

In [6]:
def Prediction(testData,prior,condition_prob) :
    numclass=prior.shape[0]
    featureNames=testData.columns 
    numclass=prior.shape[0] 
    numsample=testData.shape[0]
    featureNames=testData.columns
    post_prob=np.zeros((numsample, numclass))
    for k in range(numsample):
        prob_k=np.zeros((numclass,))
        for i in range(numclass):
            pri=prior[i]
            for feat in featureNames:
                feat_val=testData[feat][k]
                cp=condition_prob[feat]
                cp_val=cp.loc[i,feat_val]
                pri *=cp_val
            prob_k[i]=pri
        prob=prob_k/np.sum(prob_k,axis=0)
        post_prob[k,:]=prob
    return post_prob

In [7]:
prior_prob,condition_prob=Naive_Bayes(X_data,Y_data)

In [8]:
prior_prob

array([0.35714286, 0.64285714])

In [9]:
condition_prob

{'Outlook':    Overcast      Rain     Sunny
 0  0.000000  0.400000  0.600000
 1  0.444444  0.333333  0.222222,
 'Temperature':        Cool      Mild       Hot
 0  0.200000  0.400000  0.400000
 1  0.333333  0.444444  0.222222,
 'Humidity':        High    Normal
 0  0.800000  0.200000
 1  0.333333  0.666667,
 'Wind':        Weak    Strong
 0  0.400000  0.600000
 1  0.666667  0.333333}

In [10]:
test_data=[['Sunny','Cool','High','Strong']]
testData=pd.DataFrame(test_data,columns=['Outlook','Temperature','Humidity','Wind'])
testData.head()

,Outlook,Temperature,Humidity,Wind
0,Sunny,Cool,High,Strong


In [11]:
Post_prob=Prediction(testData,prior_prob,condition_prob)

In [12]:
Post_prob

array([[0.79541735, 0.20458265]])

In [13]:
if Post_prob[0][0]>Post_prob[0][1]:
    print('No')
else:
    print('Yes')

No
